In [ ]:
# To do:
# special thank you to ... 
# create markdown sections (including fMRIPrep website, nilearn website etc.) and put references
# improve comments and add print statements to facilitate human understanding
# add error tests
# 
# load afni files
# load atlas(es) for parcellation (standard and/or data-driven)
#
# Nilearn for FC (normal corr, partial corr or tangent space embedding, the latter of which has been shown to be the best?) 
# Graphpype (from Neuropycon) to calculate graph theory metrics
# Analysis of graph metrics, FC and neuropsych scores etc. using PCA, CCA and then SVM, e.g. ?????????
# 
# hypotheses and contrasts
# significance testing and multiple comparisons correction (e.g. Permutation testing or Hierarchical Bayes??)
# 
# Also check the EverNote notes !!!!!

In [ ]:
# This script is to be used on pre-processed fMRI timeseries data 

In [4]:
import glob
import os

import pandas as pd
import numpy as np
from numpy import mean
import matplotlib.pyplot as plt
import seaborn as sns

from scipy.stats import wilcoxon
from statsmodels.stats.multitest import multipletests
import statannot
from statannot import add_stat_annotation

import xlrd
import itertools

from nipype.interfaces import afni
import nibabel as nib
from nilearn import image
from nilearn import plotting
from nilearn import datasets
from nilearn import input_data
from nilearn.input_data import NiftiMapsMasker
from nilearn.input_data import NiftiLabelsMasker
from nilearn.connectome import ConnectivityMeasure


In [ ]:
# https://nipype.readthedocs.io/en/0.13.1/interfaces/generated/interfaces.afni/utils.html

In [5]:
a2n = afni.AFNItoNIFTI()
a2n.inputs.in_file = '/Users/mishodimitrov/Downloads/M014A_errts.SUBJ.tproject+tlrc.HEAD'
a2n.inputs.out_file =  'M014A_errts.SUBJ.tproject+tlrc.nii'
#a2n.cmdline 
#'3dAFNItoNIFTI -prefix M014A_errts.SUBJ.tproject+tlrc.nii M014A_errts.SUBJ.tproject+tlrc.HEAD'
res = a2n.run()

OSError: No command "3dAFNItoNIFTI" found on host Mishos-MBP.lan. Please check that the corresponding package is installed.

In [9]:
datadir = '/Users/mishodimitrov/Downloads/'
brik = nib.load(os.path.join(datadir, 'M014A_errts.SUBJ.tproject+tlrc.BRIK'))
brik_img = image.load_img(brik)
brik_img.shape

(55, 65, 55, 182)

In [10]:
head = nib.load(os.path.join(datadir, 'M014A_errts.SUBJ.tproject+tlrc.HEAD'))
head_img = image.load_img(head)
np.array_equal(head.get_fdata(), brik.get_fdata())

True

In [13]:
brik

In [15]:
brik_path = '/Users/mishodimitrov/Downloads/M014A_errts.SUBJ.tproject+tlrc.BRIK'
head_path = '/Users/mishodimitrov/Downloads/M014A_errts.SUBJ.tproject+tlrc.HEAD'
image = nib.brikhead.AFNIImage(brik_path, affine=None, header=head_path)

AFNIHeaderError: Cannot create AFNIHeader from non-AFNIHeader.

In [12]:
mean_image = image.mean_img(brik_img)
plotting.view_img(mean_image, threshold=None)

AFNIHeaderError: Cannot create AFNIHeader from nothing.